In [ ]:
from __future__ import division
import numpy as np
import cupy as cp
from keras.datasets import mnist 
from tqdm import tqdm

In [ ]:
##### parameter setting #####
cp.random.seed(0)
thr = [100, 100] 
lrF = [0.01, 0.01]  
lr = [0.1, 0.1]
b = [5, 48]  
a = [0, 0]
lamda = [0, 0] 
dropout = [0, 0]  
NumOfCatgry = 10
Tend = 255
Tmax = 256
Size_i = 28
N_ch = 1
fact = [5, 5]  
NhidenNeurons = 600
Num_neurons  = 784
Nepoch = 200
gamma = 1
best = 0  
Trbest = 0
th=[]
to=[]


In [ ]:
#### data prepration ####

(x_train, y_train), (x_test, y_test) = mnist.load_data()

Images = x_train
Labels = y_train
Images = np.array(Images)
images = []
labels = []
for i in range(len(Labels)):
    if Labels[i] < NumOfCatgry :
        images.append(np.floor(Tend-(np.dot(Images[i],Tend)/255).reshape(1,N_ch,Size_i,Size_i))) 
        labels.append(Labels[i])          


Images = x_test
Labels = y_test
Images = np.array(Images)
images_test = []
labels_test = []
for i in range(len(Labels)):
    if Labels[i] < NumOfCatgry:
        images_test.append(np.floor(Tend-(np.dot(Images[i],Tend)/255).reshape(1,N_ch,Size_i,Size_i)))
        labels_test.append(Labels[i])



images = cp.asarray(images)
images_test = cp.asarray(images_test)
labels = cp.asarray(labels)
labels_test = cp.asarray(labels_test)



In [ ]:

############## Define forward function for Fully Connected layers: ##############
def Forward_FC(FT, weight, Num_in, Num_out, init, t):
    if (init==1):
        t = cp.mgrid[0:Num_in, 0:1]
        Spikes = cp.zeros((Num_in, 1, Tmax + 1))
    else:
        t = t
        Spikes = cp.zeros((Num_in, 1, Tmax + 1))

    thre = thr[layer]/fact[layer]
    Spikes[t[0], t[1], FT.astype(int)] = 1    
    Voltage = cp.cumsum(cp.tensordot(cp.sign(weight), Spikes), 1)  
    Voltage[:,Tmax] = thre + 1
    ST = (cp.argmax(Voltage>thre,axis=1).astype(float)+1).reshape(Num_out,1) 
    ST[ST>Tmax] = Tmax

    return (ST, Voltage, t)

############## Define target function: ##############
def target_function(inp, Vinp, correct):

    target = cp.zeros([NumOfCatgry,1])
    winner = cp.argmin(inp)
    Vwin = cp.argmax(Vinp[:,inp[winner].astype(int)])        
    EQ = inp==inp[winner]  
    temp = inp[winner]

    minFiring = min(inp)             

    if minFiring == Tmax:
        target[:] = minFiring
        target[labels[IMGs[iteration]]] = minFiring 
        target = target.astype(int)
    else:
        target[:] = inp
        toChange = (inp - minFiring) < gamma
        target[toChange] = min(minFiring + gamma, Tmax)
        target[labels[IMGs[iteration]]] = minFiring

    if minFiring==Tmax:
        V=cp.argmax(Vinp[:,Tmax-3])
        if V==labels[IMGs[iteration]]:
          correct+=1
    else:
        if inp[labels[IMGs[iteration]]]==minFiring:
            correct+=1

    return(target,correct)


############## Dropout function: ##############
def DP(dp, inp, Num_in):
    if (dp>0):
        inp[cp.asarray(cp.random.permutation(Num_in)[:dp])] = Tmax+1  
    return(inp)

############## Training of the Fully Connected layers: ##############
def backward_FC(delta, inp, out, weight, lr, lamda):        
    HF = (inp<out) 
    weight -= (lr*delta)[:,cp.newaxis]*HF 
    weight -= lr*lamda*weight
    fact[layer] -= (lrF[layer]*delta[:, cp.newaxis]*HF*cp.sign(weight)).mean() 
    Delta = cp.sum((weight*delta[:,cp.newaxis])*HF,0)    
    return(weight,Delta)


In [ ]:
##### initialization #####
print('Do yo want to load and test the pretrained model? (y/n)')
pretrain= input()
if pretrain=='y':
    W0 = cp.asarray(np.load('./W0_binary.npy'))
    W1 = cp.asarray(np.load('./W1_binary.npy'))
    fact = [4.03707844, 9.06516109]
    eposchs=0
else:
    layer = 0
    W0 = (b[layer] - a[layer]) * cp.random.random_sample((NhidenNeurons,Num_neurons,1)) + a[layer]
    layer = 1
    W1 = (b[layer] - a[layer]) * cp.random.random_sample((NumOfCatgry,NhidenNeurons,1)) + a[layer]
    eposchs=Nepoch


#### Training ####
init=1
for epoch in range(eposchs):
    correct=0.0
    IMGs = cp.random.randint(0, 60000, 60000, dtype=int)  
    for iteration in tqdm(range(len(IMGs)), position=0):   

        #### Forward processing: #####
        layer = 0               
        FiringTime0 = cp.reshape(images[IMGs[iteration]],[Num_neurons,1])        
        (FiringTime1,VoltageH,th) = Forward_FC(FiringTime0, W0, Num_neurons, NhidenNeurons, init, th)
        layer = 1  
        (SpikeTime,VoltageO,to) = Forward_FC(FiringTime1, W1, NhidenNeurons, NumOfCatgry, init, to)
   
        ######## error and target time: ####
        (target, correct) = target_function(SpikeTime, VoltageO, correct)

        ##### Backward processing (Training): ######
        layer= 1   
        delta3 = cp.zeros([NumOfCatgry,1])       
        delta3 = ((target-SpikeTime)/Tmax)  
        norm = cp.linalg.norm(delta3)
        if (norm!=0):
            delta3 = delta3/norm       
        FiringTime1 = DP(dropout[layer-1], FiringTime1, NhidenNeurons)
        (W1,delta2) = backward_FC(delta3,FiringTime1,SpikeTime[:,cp.newaxis],W1,lr[layer],lamda[layer]) 

        layer= 0     
        norm = cp.linalg.norm(delta2)
        if (norm!=0):
            delta2 = delta2/norm       
        FiringTime0 = DP(dropout[layer-1], FiringTime0, Num_neurons) 
        (W0,delta1) = backward_FC(delta2,FiringTime0,FiringTime1[:,cp.newaxis],W0,lr[layer],lamda[layer]) 
       
        init = 0                 

 
        ######## TEST: ########
    correct=0
    for iteration in tqdm(range(len(images_test)), position=0):
        ############# Forward pass: ##############
        layer = 0               
        FiringTime0 = cp.reshape(images_test[iteration],[Num_neurons,1])        
        (FiringTime1,VoltageH,th) = Forward_FC(FiringTime0, W0, Num_neurons, NhidenNeurons, init, th)

        layer = 1  
        (SpikeTime,VoltageO,to) = Forward_FC(FiringTime1, W1, NhidenNeurons, NumOfCatgry, init, to)

        ############## Decision making ##############
        winner = cp.argmin(SpikeTime)
        Vwin = cp.argmax(VoltageO[:,SpikeTime[winner].astype(int)])        
        EQ = SpikeTime==SpikeTime[winner]  
        temp = SpikeTime[winner]
        minFiring = min(SpikeTime)
        maxFiring = max(SpikeTime)             

        if minFiring==Tmax:
            V=np.argmax(VoltageO[:,Tmax-3])
            if V==labels_test[iteration]:
              correct+=1
        else:
            if SpikeTime[labels_test[iteration]]==minFiring:
                correct+=1             

    print('epoch = ', epoch, 'Test accuracy= ', correct/len(images_test), 'Best Accuracy= ', best)

    if (correct/len(images_test)>=best):
        best = correct/len(images_test)
        np.save('./W0_binary_best.npy',cp.asnumpy(W0))
        np.save('./W1_binary_best.npy',cp.asnumpy(W1))
    
    
######## Fibnal TEST result: ########
correct=0
for iteration in tqdm(range(len(images_test)), position=0):#progressbar.progressbar(range(len(images_test))):   #for iteration in range(len(images_test)): 
    ############# Forward pass: ##############
    layer = 0               
    FiringTime0 = cp.reshape(images_test[iteration],[Num_neurons,1])        
    (FiringTime1,VoltageH,th) = Forward_FC(FiringTime0, W0, Num_neurons, NhidenNeurons, init, th)

    layer = 1  
    (SpikeTime,VoltageO,to) = Forward_FC(FiringTime1, W1, NhidenNeurons, NumOfCatgry, init, to)

    ############## Decision making ##############
    winner = cp.argmin(SpikeTime)
    Vwin = cp.argmax(VoltageO[:,SpikeTime[winner].astype(int)])        
    EQ = SpikeTime==SpikeTime[winner]  
    temp = SpikeTime[winner]
    minFiring = min(SpikeTime)
    maxFiring = max(SpikeTime)             

    if minFiring==Tmax:
        V=np.argmax(VoltageO[:,Tmax-3])
        if V==labels_test[iteration]:
          correct+=1
    else:
        if SpikeTime[labels_test[iteration]]==minFiring:
            correct+=1             

print('Final Test accuracy= ', correct/len(images_test))
